In [10]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import keras
import tensorflow as tf
import keras.backend as K
from keras.utils import to_categorical
from keras import metrics
from keras.models import Model, load_model

from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Maximum, concatenate
from keras.layers.core import Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose,Conv3D,Conv3DTranspose

from keras.layers.pooling import MaxPooling2D,MaxPooling3D

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam

from skimage.io import imread, imshow
from skimage.transform import resize

from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping


import nibabel as nib
from tqdm import tqdm

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brats-2021-task1/BraTS2021_00495.tar
/kaggle/input/brats-2021-task1/BraTS2021_Training_Data.tar
/kaggle/input/brats-2021-task1/BraTS2021_00621.tar


In [333]:
###Libraries and imports
import numpy as np
import math
import random
import os
import shutil
import gzip
import glob
import gc
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import tarfile
import PIL
import scipy.misc
import skimage
import nibabel as nib

from PIL import Image
from tqdm import tqdm
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers 
from keras import backend as K
from sklearn.model_selection import train_test_split
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from skimage.io import imread
from scipy import ndimage as ndi
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from glob import glob

from tensorflow import keras
from tensorflow.keras import backend as K 
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (
    Activation,
    Conv3D,
    Conv3DTranspose,
    MaxPooling3D,
    UpSampling3D,
)
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers, regularizers


In [3]:
# open the tar file for reading
tar = tarfile.open('/kaggle/input/brats-2021-task1/BraTS2021_Training_Data.tar', 'r')

# extract all files and directories
tar.extractall()


In [11]:
# get a list of all directories in the current folder
directories = [name for name in os.listdir() if os.path.isdir('/kaggle/working/')]

nums = [str(x.split('_')[1]) for x in directories]

directories.remove('__notebook_source__.ipynb')
directories.remove('.virtual_documents')
directories.remove('.DS_Store')

for i in directories:
    y = i.split('_')[1]
    if len(y)<5 :
        print("Here: ",i)

# sort the directories based on the number in the name
# sorted_directories = sorted(directories, key=lambda x: int(x.split('_')[1]) if x.split('_')[1].isdigit() else float('inf'))

sorted_directories = sorted(directories, key=lambda x: int(x.split('_')[1]))

print(len(sorted_directories))
# print the sorted list of directories
print("Directories in BraTS2021_Training_Data.tar (sorted):")
for directory in sorted_directories[:5]:
    print(directory)

1251
Directories in BraTS2021_Training_Data.tar (sorted):
BraTS2021_00000
BraTS2021_00002
BraTS2021_00003
BraTS2021_00005
BraTS2021_00006


In [355]:
"""
Dice = (2*|X & Y|)/ (|X|+ |Y|)
      =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
ref: https://arxiv.org/pdf/1606.04797v1.pdf

"""


def dice_coef(y_true, y_pred, epsilon=0.00001):
    axis = (0,1,2,3)
    dice_numerator = 2. * K.sum(K.abs(y_true * y_pred), axis=axis) + epsilon
    dice_denominator = K.sum(y_true*y_true, axis=axis) + K.sum(y_pred*y_pred, axis=axis) + epsilon
    return K.mean((dice_numerator)/(dice_denominator))
#     return 2*(dice_numerator)/(dice_denominator)


# def dice_coef(y_true, y_pred, epsilon=0.00001):
#     A = y_true
#     B = y_pred
    
#     smooth = 1e-7
    
#     intersection = np.sum(A * B, axis=(0,1,2,3))
#     sum_A = np.sum(A, axis=(0,1,2,3))
#     sum_B = np.sum(B, axis=(0,1,2,3))
    
#     dice = (2. * intersection + smooth) / (sum_A + sum_B + smooth)
    
#     return dice



# define per class evaluation of dice coef

def dice_coef_necrotic(y_true, y_pred, epsilon=0.000001): ## NON-ENHANCING tumor CORE
    intersection = K.sum(K.abs(y_true[:,:,:,1] * y_pred[:,:,:,1]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,1])) + K.sum(K.square(y_pred[:,:,:,1])) + epsilon)

def dice_coef_edema(y_true, y_pred, epsilon=0.000001):
    intersection = K.sum(K.abs(y_true[:,:,:,2] * y_pred[:,:,:,2]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,2])) + K.sum(K.square(y_pred[:,:,:,2])) + epsilon)

def dice_coef_enhancing(y_true, y_pred, epsilon=0.000001):
    intersection = K.sum(K.abs(y_true[:,:,:,3] * y_pred[:,:,:,3]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,3])) + K.sum(K.square(y_pred[:,:,:,3])) + epsilon)



# Computing Precision
# Precision: TP / (TP + FP)
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

    
# Computing Sensitivity  
# Sensitivity or recall: TP / (TP + FN)
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


# Computing Specificity
# Specificity: TN / (TN + FP)
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [356]:
def conv_block(input_mat,num_filters,kernel_size,batch_norm):
  X = Conv3D(num_filters,kernel_size=(kernel_size,kernel_size,kernel_size),strides=(1,1,1),padding='same')(input_mat)
  if batch_norm:
    X = BatchNormalization()(X)
  
  X = Activation('relu')(X)

  X = Conv3D(num_filters,kernel_size=(kernel_size,kernel_size,kernel_size),strides=(1,1,1),padding='same')(X)
  if batch_norm:
    X = BatchNormalization()(X)
  
  X = Activation('relu')(X)
  
  return X


def Unet_3d(input_img, n_filters = 16, dropout = 0.2, batch_norm = True):

  c1 = conv_block(input_img,n_filters,3,batch_norm)
  p1 = MaxPooling3D(pool_size=(2, 2, 2), strides=2)(c1)
  p1 = Dropout(dropout)(p1)
  
  c2 = conv_block(p1,n_filters*2,3,batch_norm);
  p2 = MaxPooling3D(pool_size=(2,2,2) ,strides=2)(c2)
  p2 = Dropout(dropout)(p2)

  c3 = conv_block(p2,n_filters*4,3,batch_norm);
  p3 = MaxPooling3D(pool_size=(2,2,2) ,strides=2)(c3)
  p3 = Dropout(dropout)(p3)
  
  c4 = conv_block(p3,n_filters*8,3,batch_norm);
  p4 = MaxPooling3D(pool_size=(2,2,2) ,strides=2)(c4)
  p4 = Dropout(dropout)(p4)
  
  c5 = conv_block(p4,n_filters*16,3,batch_norm);

  u6 = Conv3DTranspose(n_filters*8, (3,3,3), strides=(2, 2, 2), padding='same')(c5);
  u6 = concatenate([u6,c4]);
  c6 = conv_block(u6,n_filters*8,3,batch_norm)
  c6 = Dropout(dropout)(c6)
  u7 = Conv3DTranspose(n_filters*4,(3,3,3),strides = (2,2,2) , padding= 'same')(c6);

  u7 = concatenate([u7,c3]);
  c7 = conv_block(u7,n_filters*4,3,batch_norm)
  c7 = Dropout(dropout)(c7)
  u8 = Conv3DTranspose(n_filters*2,(3,3,3),strides = (2,2,2) , padding='same')(c7);
  u8 = concatenate([u8,c2]);

  c8 = conv_block(u8,n_filters*2,3,batch_norm)
  c8 = Dropout(dropout)(c8)
  u9 = Conv3DTranspose(n_filters,(3,3,3),strides = (2,2,2) , padding='same')(c8);

  u9 = concatenate([u9,c1]);

  c9 = conv_block(u9,n_filters,3,batch_norm)
  outputs = Conv3D(4, (1, 1,1), activation='softmax')(c9)

  model = Model(inputs=input_img, outputs=outputs)

  return model



def standardize(img_data):
    mean = np.mean(img_data)
    std = np.std(img_data)
    return (img_data - mean) / std

#     standardized_image = np.zeros(image.shape)
#     for z in range(image.shape[2]):
#         image_slice = image[:,:,z]

#         # subtract the mean from image_slice
#         centered = image_slice - np.mean(image_slice)

#         # divide by the standard deviation (only if it is different from zero)
#         if(np.std(centered)!=0):
#           centered = centered/np.std(centered) 

#         # update  the slice of standardized image
#         # with the scaled centered and scaled image
#         standardized_image[:, :, z] = centered
#     return standardized_image


def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)



In [357]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the callback to save model checkpoints

checkpoint_callback = ModelCheckpoint(
    filepath='my_model_checkpoint_{epoch}.h5',
    save_weights_only=False,
    verbose=1
)

# callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.000001, verbose=1),    checkpoint,    SaveEpochs(),    csv_logger]
callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.000001, verbose=1), checkpoint_callback]

In [358]:
# path = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'
path = '/kaggle/working/'

def imageLoader(path, folders, batch_size=1, Req=None):

    L = len(folders)
    
    data = np.zeros((240,240,155,4))
    image_data2=np.zeros((240,240,155))

    #keras needs the generator infinite, so we will use while true  
    while True:

        image_num = 0
        batch_end = batch_size

        while image_num < L: # 369 images in total
            limit = min(batch_end, 1251)
                
            x = folders[image_num]
#             if(Req):
#                 print("Here "+ Req + " " + x)
            
            folder_path = path + '/' + x;
            modalities = os.listdir(folder_path)
            modalities.sort()
            
#             print(x)
#             print(modalities)
            
            
            w = 0
#             BraTS2021_00618_flair.nii.gz', 'BraTS2021_00618_seg.nii.gz', 'BraTS2021_00618_t1.nii.gz', 'BraTS2021_00618_t1ce.nii.gz', 'BraTS2021_00618_t2.nii.gz
            
            for j in range(len(modalities)):
              image_path = folder_path + '/' + modalities[j]
#               print(image_path[-10:-1] + image_path[-1])
              if(image_path[-10:-1] + image_path[-1] == 'seg.nii.gz'):
                img = nib.load(image_path);
                image_data2 = img.get_fdata()
                image_data2 = np.asarray(image_data2)
              else:
                img = nib.load(image_path);
                image_data = img.get_fdata()
#                 print("1: ",np.unique(image_data))
                image_data = np.asarray(image_data)
#                 print("2: ",np.unique(image_data))
                image_data = standardize(image_data)
#                 print("3: ",np.unique(image_data))
                data[:,:,:,w] = image_data
                w = w+1
            
            reshaped_data=data[56:184,80:208,13:141,:]
    
            reshaped_image_data2=image_data2[56:184,80:208,13:141]
        
        
            reshaped_data=reshaped_data.reshape(1,128,128,128,4)
            reshaped_image_data2=reshaped_image_data2.reshape(1,128,128,128)
            reshaped_image_data2[reshaped_image_data2==4] = 3
            hello = reshaped_image_data2.flatten()
            
            
#             class_weights = class_weight.compute_class_weight('balanced',classes = np.unique(hello),y = hello)

            reshaped_image_data2 = to_categorical(reshaped_image_data2, num_classes = 4)
            
            
                       
            X = reshaped_data
            Y = reshaped_image_data2

            yield (X,Y)    

            image_num += batch_size   
            batch_end += batch_size

In [359]:
from sklearn.model_selection import train_test_split

print(sorted_directories[-5:])

print(len(sorted_directories))

['BraTS2021_01662', 'BraTS2021_01663', 'BraTS2021_01664', 'BraTS2021_01665', 'BraTS2021_01666']
1251


In [360]:
# Load the NIfTI file
img = nib.load('/kaggle/working/BraTS2021_00801/BraTS2021_00801_t1.nii.gz')

# Get the first volume of the image data (assuming 4D data)
data = img.get_fdata()

In [361]:
data.shape

(240, 240, 155)

In [362]:
train_folders, test_folders  = train_test_split(sorted_directories, test_size=0.25, random_state=42)

# val_folders, test_folders = train_test_split(test_val_folders, test_size=0.5, random_state=42)

# print the results
# print('Test_Val Folders:', len(test_val_folders))
print('Train Folders:', len(train_folders))
print('Test Folders:', len(test_folders))
# print('Val Folders:', len(val_folders))

Train Folders: 938
Test Folders: 313


In [363]:
batch_size = 1

path = '/kaggle/working/'

# train_img_datagen = imageLoader(path, train_folders, batch_size)
# val_img_datagen = imageLoader(path, val_folders, batch_size)
# test_img_datagen = imageLoader(path, test_folders, batch_size)

In [364]:
# X,y = train_img_datagen.__next__()
# X.shape, y.shape

In [365]:
# !pip install segmentation-models-3D

In [366]:
import segmentation_models_3D as sm

In [373]:
def combined_loss(y_true, y_pred, alpha=0.5):
    # Compute the DC loss
    dc_loss = dice_coef_loss(y_true, y_pred)
#     print(dc_loss)
    
    # Compute the binary cross-entropy (BCE) loss
#     bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    bce = K.categorical_crossentropy(y_true, y_pred)
    bce_loss = K.mean(bce, axis=(1, 2, 3))
#     print(bce_loss)
    
    # Combine the MSE and BCE losses with a weighting factor
    combined_loss = alpha * dc_loss + (1 - alpha) * bce_loss
    
    return combined_loss

In [374]:
input_img = Input((128,128,128,4))
model = Unet_3d(input_img)


metrics = ['accuracy', sm.metrics.IOUScore(threshold=0.5),dice_coef, precision, sensitivity, specificity, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing]
model.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=0.001, decay=0.0001), loss=combined_loss, metrics=metrics)
# optimizer=keras.optimizers.legacy.Adam(learning_rate=0.001, decay=0.0001)

In [375]:
from sklearn.model_selection import KFold

In [376]:
k = 5
kf = KFold(n_splits=k, shuffle=True)

In [377]:
lyst = []

In [378]:
for fold, (train_indices, val_indices) in enumerate(kf.split(train_folders)):
    print('Fold:', fold)
    batch_size = 1
#     print(train_indices, val_indices)
    
#     print(max(train_indices),max(val_indices))

    train_data = [sorted_directories[x] for x in train_indices]
    val_data = [sorted_directories[x] for x in val_indices]
    
#     print(len(train_data))
#     print(len(val_data))
    
    train_img_datagen = imageLoader(path, train_data, batch_size)
    val_img_datagen = imageLoader(path, val_data, batch_size)

#     model.fit(train_data, epochs=10, validation_data=val_data)
    model.fit(train_img_datagen, 
                  epochs=10, 
                  steps_per_epoch=(len(train_folders)//batch_size),
                  validation_data=val_img_datagen,
                  validation_steps=(len(val_folders)//batch_size),
                  verbose=1,
                  callbacks=callbacks)

    scores = model.evaluate(val_data)
    lyst.append(scores)
    print('Validation accuracy:', scores[1])

Fold: 0
Epoch 1/10
938/938 [==============================] - ETA: 0s - loss: 0.2484 - accuracy: 0.9649 - iou_score: 0.5830 - dice_coef: 0.6838 - precision: 0.9716 - sensitivity: 0.9329 - specificity: 0.9944 - dice_coef_necrotic: 0.9536 - dice_coef_edema: 0.9531 - dice_coef_enhancing: 0.9526
Epoch 1: saving model to my_model_checkpoint_1.h5
938/938 [==============================] - 1496s 2s/step - loss: 0.2484 - accuracy: 0.9649 - iou_score: 0.5830 - dice_coef: 0.6838 - precision: 0.9716 - sensitivity: 0.9329 - specificity: 0.9944 - dice_coef_necrotic: 0.9536 - dice_coef_edema: 0.9531 - dice_coef_enhancing: 0.9526 - val_loss: 0.2809 - val_accuracy: 0.9721 - val_iou_score: 0.4614 - val_dice_coef: 0.5608 - val_precision: 0.9729 - val_sensitivity: 0.9715 - val_specificity: 0.9910 - val_dice_coef_necrotic: 0.9992 - val_dice_coef_edema: 0.9990 - val_dice_coef_enhancing: 0.9990 - lr: 0.0010
Epoch 2/10
938/938 [==============================] - ETA: 0s - loss: 0.1277 - accuracy: 0.9853 - iou

KeyboardInterrupt: 

In [231]:
history=model.fit(train_img_datagen, 
                  epochs=20, 
                  steps_per_epoch=(len(train_folders)//batch_size),
                  validation_data=val_img_datagen,
                  validation_steps=(len(val_folders)//batch_size),
                  verbose=1,
                  callbacks=callbacks)

Epoch 1/20
269/938 [=======>......................] - ETA: 14:48 - loss: 0.0305 - accuracy: 0.9604 - iou_score: 0.5148 - dice_coef: 1.2039 - precision: 0.9703 - sensitivity: 0.9264 - specificity: 0.9923 - dice_coef_necrotic: 0.9645 - dice_coef_edema: 0.9627 - dice_coef_enhancing: 0.9625

KeyboardInterrupt: 